In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Sentiment Analysis

In [3]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [4]:

bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [5]:
# Fonction pour l'analyse de sentiment en utilisant l'API Messages de Claude 3
def analyze_sentiment(text):
    try:
        # Préparation de la requête pour l'API Messages
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        # Affichage de la réponse brute pour diagnostic
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Vérification de la structure de la réponse et extraction du texte
        content = response_body.get("content")
        if isinstance(content, list):
            sentiment_analysis = " ".join([part["text"] for part in content if "text" in part])
        else:
            sentiment_analysis = content  # Directement utiliser le contenu si c'est une chaîne
        
        return sentiment_analysis or "Pas de résultat détecté"
    except Exception as e:
        print("Erreur :", e)
        return None

In [6]:
# # Exemple d'utilisation
# text_to_analyze = "J'aime pas le fromage."
# result = analyze_sentiment(text_to_analyze)
# print("Analyse de sentiment :", result)

## RAG Agent

In [7]:
agent_id = os.getenv("AGENT_ID")
agent_alias = os.getenv("AGENT_ALIAS")

In [8]:

runtime_client=boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  # Facultatif
),


In [9]:
runtime_client[0]

In [10]:
import uuid
session_id = str(uuid.uuid4())

# Function to call the agent with a user prompt
def call_agent_with_prompt(user_prompt,agent_alias):
    try:
        # Invoke the agent without invalid parameters
        response = runtime_client[0].invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias,  # Include this if you have an agent alias
            sessionId=session_id,
            inputText=user_prompt,
            #enableTrace=True
        )
        event_stream = response.get('completion')
        response_text = ''

        # Parcourir l'EventStream pour obtenir la réponse
        for event in event_stream:
            # Chaque événement est un dictionnaire contenant les données de l'événement
            # Pour la génération de texte, les données sont sous la clé 'chunk'
            if 'chunk' in event:
                chunk = event['chunk']
                # La clé 'bytes' contient le texte généré
                content = chunk.get('bytes', b'').decode('utf-8')
                response_text += content
            else:
                print("Événement non traité :", event)

        return response_text

    except Exception as e:
        print("Erreur lors de l'appel à l'agent :", e)
        return None

In [11]:
# # Exemple d'appel avec un *prompt* utilisateur
# user_prompt = "Peux tu analyser une analyse fondamentale de Accenture ?"
# response = call_agent_with_prompt(user_prompt,agent_alias)
# print("Réponse de l'agent :", response)


## Agent + macro + sentiment

In [12]:
import yfinance as yf
import pandas as pd

In [13]:
ticker = yf.Ticker("AAPL")

Sentiment

In [14]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire le texte principal de l'article
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraction des paragraphes de l'article
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    return article_text

In [15]:

def analyze_subject_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 100,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Donne uniquement sujet principal de cet article ainsi que le sentiment du text associé sans explicatioN: \"{text}\""}
                ]
            })
        )
        
        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        subject_analysis = content[0]["text"] if content else "Pas de sujet détecté"
        return subject_analysis
    
    except Exception as e:
        print("Erreur d'analyse du sujet :", e)
        return None



In [16]:

# Fonction principale pour obtenir les informations d'actualité avec sujet et sentiment
def get_news_with_sentiment(ticker):
    

    news = ticker.news
    sentiment_output = ''
    
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        
        # Extraire le texte de l'article
        article_text = extract_article_text(link)
        
        # Analyser le sujet et le sentiment
        subject_sentiment = analyze_subject_sentiment(article_text)
        
        sentiment_output += title + subject_sentiment
        

    return sentiment_output

In [17]:
sentiment_output = get_news_with_sentiment(ticker)

In [18]:
sentiment_output

"iPhone 16’s Impact on Apple (AAPL)’s Market Share: A Mixed BagSujet principal: Retard des données boursières et tendances des actions d'IA à surveiller en novembre, notamment Apple Inc. (AAPL).\n\nSentiment: Neutre/InformatifBuffett Keeps Cutting Apple Stake, With Value Down 60% This YearSujet principal : Berkshire Hathaway a considérablement réduit sa participation dans Apple au troisième trimestre 2023.\n\nSentiment du texte : Neutre, le texte relate de manière factuelle les ventes d'actions Apple par Berkshire Hathaway et fournit des détails sur les résultats financiers du troisième trimestre.3 Things Apple Doesn't Want to Talk AboutSujet principal: Problème technique en cours de résolution\n\nSentiment: Rassurant, positifBig Pension Sold Palantir, Apple, and Nvidia Stock. It Bought AMD.Sujet principal: Mise à jour sur un problème technique en cours\n\nSentiment du texte: RassurantApple Intelligence Products Could Make Up For Apple’s (AAPL) iPhone WeaknessSujet principal : Les diff

Macro

In [19]:
def caption_summary(text,max_token):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_token,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Tu es un macro économiste, rédige un résumé des points clés  macro économique des différents sujets, avec détails (ex chiffres) : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)

In [20]:

def get_macro_news() :
    tickers = {
        "^GSPC": "S&P 500 - Indice Boursier",
        "DX-Y.NYB": "Indice du Dollar Américain",
        "^TNX": "Taux des Obligations à 10 ans",
        "GC=F": "Prix de l'Or",
        "CL=F": "Prix du Pétrole (WTI)",
        "^VIX": "VIX - Volatilité des Marchés"
    }
    
    output_text = ""
    for ticker_symbol, description in tickers.items():
        ticker = yf.Ticker(ticker_symbol)
        news_items = ticker.news
        
        for item in news_items[:2]:
            
            text_article = extract_article_text(item["link"])
            if text_article != "Thank you for your patience. Our engineers are working quickly to resolve the issue." :
                
                text = caption_summary(text_article,150)
                
                output_text += text
    caption_macro = caption_summary(output_text,max_token=300)
    return caption_macro

In [21]:
output_macro = get_macro_news()

In [22]:
output_macro

"Voici un résumé des principaux points macroéconomiques avec des détails chiffrés:\n\nCroissance économique:\n- Le PIB des États-Unis a augmenté de 2,8% au troisième trimestre 2024, légèrement en-dessous des prévisions de 3,1%. \n- La consommation des ménages, moteur de l'économie américaine, a progressé de 3,7%, son rythme le plus élevé depuis début 2023, contribuant fortement à la croissance du PIB.\n\nMarché du travail:\n- Le secteur privé américain a créé 230 000 nouveaux emplois en octobre 2024, dépassant les attentes des analystes qui tablaient sur 200 000 créations d'emplois.\n- Le taux de chômage est resté stable à 3,7%, proche du plein emploi.\n\nInflation: \n- L'indice des prix à la consommation (IPC) a augmenté de 0,3% en octobre sur un mois et de 2,5% sur un an.\n- L'inflation sous-jacente, excluant les composantes volatiles alimentaires et énergétiques, s'est"

In [ ]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Réalise le rapport sur AAPL en détaillant les 5 parties ?"
promt = user_prompt + "Context macro suppélementaire :" + output_macro + "Sujets et sentiments actuelles autour de l'entreprise :" + sentiment_output
response = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response)

Réponse de l'agent : Voici le rapport détaillé en 5 parties sur l'entreprise AAPL :

1. Présentation générale de l'entreprise :
- AAPL est une entreprise américaine spécialisée dans les produits électroniques grand public, notamment les smartphones, ordinateurs, tablettes et accessoires.
- Elle opère dans le secteur technologique et fait partie de l'industrie des biens de consommation électroniques.
- AAPL emploie environ 150 000 personnes à temps plein dans le monde.

2. Indicateurs financiers clés :
- AAPL a généré un chiffre d'affaires de 391,04 milliards de dollars en 2022, avec une marge opérationnelle de 31,17%.
- Son bénéfice avant intérêts, impôts, dotations et amortissements (EBITDA) s'élève à 134,66 milliards de dollars.
- La société dispose d'une trésorerie nette de 65,17 milliards de dollars et d'un endettement total de 106,63 milliards de dollars.
- Le ratio cours/valeur comptable est de 59,17 et le ratio de distribution des dividendes est de 16,12%.

3. Analyse de la perf

## Agent-macro

In [23]:
agent_alias = os.getenv("AGENT_MACRO_ALIAS")

runtime_client=boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  # Facultatif
),

In [25]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Réalise l'analyse macroéconimque en détail ?"
promt = user_prompt + "Context macro suppélementaire :" + output_macro 
response_macro = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response_macro)

Réponse de l'agent : Dans l'ensemble, le contexte macroéconomique aux États-Unis est plutôt favorable, avec une croissance économique modérée, un marché du travail dynamique et une inflation maîtrisée. Cependant, le léger ralentissement de la croissance du PIB et la concurrence sur un marché du travail tendu pourraient représenter des défis pour l'entreprise. Elle devra donc rester vigilante et s'adapter pour tirer parti des opportunités et limiter les risques.


## Agent profil

In [26]:
agent_alias = os.getenv("AGENT_PROFIL_ALIAS")

In [27]:
ticker_name = "AAPL" # mettre la variable choisi dans la liste
user_prompt = f"Présente l'entreprise {ticker_name} ?"
promt = user_prompt 
response_profil = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response_profil)

Réponse de l'agent : 

Voici une analyse financière et sectorielle de l'entreprise Apple Inc. (AAPL) :

Apple Inc. est une entreprise américaine du secteur technologique, plus précisément de l'industrie des produits électroniques grand public. Ses principaux produits sont les smartphones iPhone, les ordinateurs personnels Mac, les tablettes iPad, ainsi que divers accessoires et services tels qu'AppleCare, Apple Music, Apple TV+, Apple Pay, etc.

Principaux indicateurs financiers clés :
- Chiffre d'affaires : 30,7 milliards de dollars en 2024 (prévision), en hausse de 0,8% par rapport à 2023
- Bénéfice net : 712,3 millions de dollars en 2024 (prévision), en hausse de 16,8% par rapport à 2023
- Marge bénéficiaire nette : 2,3% en 2024 (prévision)
- Rendement du dividende : 0,45%
- Ratio de distribution : 16,12%
- Ratio cours/bénéfice (P/E) historique : 36,60
- Ratio cours/bénéfice (P/E) prospectif : 26,73

Au niveau sectoriel, Apple évolue dans un marché très concurrentiel des produits él

## Agent Financial


In [28]:
agent_alias = os.getenv("AGENT_FINANCE_ALIAS")

ticker_name = "AAPL" # mettre la variable choisi dans la liste
user_prompt = f"Réalise l'analyse de l'état financère de l'entreprise {ticker_name} ?"
promt = user_prompt 
response_fin = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response_fin)

Réponse de l'agent : Voici une analyse financière détaillée de l'entreprise AAPL :

Secteur et activité :
AAPL est une entreprise américaine du secteur technologique, spécialisée dans la conception, la fabrication et la commercialisation de smartphones, d'ordinateurs personnels, de tablettes, de produits connectés et d'accessoires. Elle opère également dans les services numériques et de paiement.

Situation financière :
- AAPL a une forte position de trésorerie avec 65,17 milliards de dollars de trésorerie totale et 4,31 dollars de trésorerie par action.
- Son EBITDA s'élève à 134,66 milliards de dollars, témoignant d'une bonne rentabilité opérationnelle.
- Cependant, l'entreprise a un niveau d'endettement élevé avec 106,63 milliards de dollars de dette totale et un ratio dette/capitaux propres de 187%.
- Les ratios de liquidité sont corrects avec un ratio courant de 0,87 et un ratio rapide de 0,75.
- La rentabilité est satisfaisante avec un rendement des actifs de 21,5% et un rendemen

## Agent sentiment

In [29]:
agent_alias = os.getenv("AGENT_SENTIMENT_ALIAS")

ticker_name = "AAPL" # mettre la variable choisi dans la liste
user_prompt = f"Réalise l'analyse du sentiment de marché de l'entreprise {ticker_name} avec comme context les dernières actualité et leurs sentiments?"
promt = user_prompt + "Context actualité et sentiments : " + sentiment_output
response_sent = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response_sent)

Réponse de l'agent : 

Après avoir analysé les dernières actualités et les indicateurs clés de performance d'Apple (AAPL), voici mon évaluation du sentiment du marché concernant cette entreprise :

Dans l'ensemble, le sentiment du marché semble être plutôt neutre à légèrement positif concernant Apple. Bien qu'il y ait eu quelques défis, comme la baisse des ventes d'iPhone et la réduction de la participation de Berkshire Hathaway, les articles montrent également des aspects positifs, comme les attentes pour les futurs produits d'intelligence artificielle d'Apple et l'acquisition de Pixelmator.

D'un point de vue financier, Apple reste une entreprise très performante, avec une forte rentabilité, une trésorerie importante et une position de leader dans son secteur. Ses indicateurs clés de performance, comme le chiffre d'affaires, la marge opérationnelle et le free cash flow, sont solides. Cependant, ses perspectives de croissance des bénéfices sont négatives à court terme.

Malgré ces déf

## Agent Risk

In [30]:
agent_alias = os.getenv("AGENT_RISK_ALIAS")

ticker_name = "AAPL" # mettre la variable choisi dans la liste
user_prompt = f"Réalise l'analyse des risques et opportunité de l'entreprise {ticker_name} selon le context macroéconomic et la valeur beta kpi"
promt = user_prompt + "Context macroeconomic : " + output_macro
response_risk = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response_risk)

Réponse de l'agent : 

En résumé, l'analyse du contexte macroéconomique et de la valeur bêta de l'entreprise AAPL fait ressortir les éléments suivants :

Risques :
- Volatilité du titre AAPL plus élevée que la moyenne du marché, avec une valeur bêta de 1,239. Cela signifie un risque plus important pour les investisseurs.
- Dépendance importante de l'entreprise à la demande des consommateurs, qui peut être volatile.
- Forte exposition internationale, rendant AAPL sensible aux fluctuations macroéconomiques.

Opportunités :
- Contexte macroéconomique globalement favorable aux États-Unis, avec une croissance économique soutenue, un marché du travail dynamique et une inflation modérée.
- La consommation des ménages, moteur de l'économie américaine, progresse à un rythme élevé, ce qui devrait bénéficier aux ventes de l'entreprise AAPL.





## Agent résumé

In [31]:
tot_reponse = response_risk + response_profil + response_macro + response_fin + response_sent

In [32]:
agent_alias = os.getenv("AGENT_RESUME_ALIAS")

ticker_name = "AAPL" # mettre la variable choisi dans la liste
user_prompt = f"Réalise un récapitulatif et un conseil de stratégie vis à vis de l'entreprise {ticker_name} "
promt = user_prompt + "Context :" + tot_reponse
response = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response)

Réponse de l'agent : Voici une analyse financière et sectorielle détaillée d'Apple Inc. (AAPL) :

Situation financière :
- Apple dispose d'une très solide position de trésorerie avec 65,17 milliards de dollars de trésorerie totale et 4,31 dollars de trésorerie par action.
- Son EBITDA s'élève à 134,66 milliards de dollars, témoignant d'une bonne rentabilité opérationnelle.
- Cependant, l'entreprise a un niveau d'endettement élevé avec 106,63 milliards de dollars de dette totale et un ratio dette/capitaux propres de 187%.
- Les ratios de liquidité sont corrects avec un ratio courant de 0,87 et un ratio rapide de 0,75.
- La rentabilité est satisfaisante avec un rendement des actifs de 21,5% et un rendement des capitaux propres de 157,4%.

Performances :
- Le chiffre d'affaires s'élève à 391,04 milliards de dollars, en croissance de 6,1% sur un an.
- Cependant, les bénéfices ont baissé de 33,5% sur la même période.
- Les marges brutes sont élevées à 46,2% mais les marges opérationnelles s

## AGENT Chat

In [ ]:
agent_alias = os.getenv("AGENT_CHAT_ALIAS")
user_prompt = "" #From chat
promt = user_prompt + "Context macro  :" + output_macro + "Sujets et sentiments actuelles autour de l'entreprise :" + sentiment_output
response = call_agent_with_prompt(promt,agent_alias)
print("Réponse de l'agent :", response)